In [25]:
import os
from os.path import join as jp
from itertools import product

import pandas as pd
import numpy as np
import nibabel as nib
from tqdm import tqdm_notebook as tqdm

from dpipe.io import load
from dpipe.im.visualize import slice3d

In [26]:
img = 'images'
msk = 'masks'
root = '/path/cc359'

path_img = jp(root, img)
path_msk = jp(root, msk)

fnames = sorted(os.listdir(path_img))

cites = ('siemens', 'ge', 'philips')
tsls = ('15', '3')

In [27]:
pairs2fold = {p[1]: p[0] for p in list(enumerate(list(product(cites, tsls))))}

In [28]:
def fname2info(fname):
    f = fname.strip('.nii.gz')
    mask_fname = f + '_ss.nii.gz'
    _id, cite, tsl, *_ = f.split('_')
    return _id, cite, tsl, mask_fname

In [32]:
records = []
for fname in tqdm(fnames):
    input_path = jp(path_img, fname)
    data = nib.load(input_path)
    _id, cite, tsl, mask_fname = fname2info(fname)
    data = nib.load(input_path)
    vs = data.header['pixdim'][1:4]
    scan = data.get_data()    
    shape = scan.shape
    record = {
        'id': _id,
        'MRI': jp(img, fname),
        'brain_mask': jp(msk, mask_fname),
        'fold': pairs2fold[(cite, tsl)],
        'tomograph_model': cite,
        'tesla_value': tsl,
        'x': vs[0],
        'y': vs[1],
        'z': vs[2],
        'shape': shape,
        # 'shape:': str(shape[0]) + ', ' + str(shape[1]) + ', ' + str(shape[2]),
    }
    records.append(record)
    
meta = pd.DataFrame.from_records(records, index='id')
meta.to_csv(jp(root, 'meta.csv'), index_label='id')

/tmp/ipykernel_45669/3068803342.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for fname in tqdm(fnames):


  0%|          | 0/359 [00:00<?, ?it/s]

/tmp/ipykernel_45669/3068803342.py:8: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  scan = data.get_data()


In [16]:
img = data.get_data()

/tmp/ipykernel_45669/1045731120.py:1: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  img = data.get_data()


In [17]:
img.shape

(182, 218, 182)